## Imports

In [1]:
%tensorflow_version 1.x
import tensorflow as tf
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np
## for data
import json
import pandas as pd
import numpy as np
## for plotting
import matplotlib.pyplot as plt
import seaborn as sns
## for processing
import re
import nltk
## for bag-of-words
from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing
## for explainer
# from lime import lime_text
## for word embedding
import gensim
import gensim.downloader as gensim_api
## for deep learning
from tensorflow.keras import models, layers, preprocessing as kprocessing
from tensorflow.keras import backend as K

TensorFlow 1.x selected.


## Connect to GoogleDrive

In [2]:
from google.colab import drive
drive.mount('/gdrive')
!cd /gdrive/MyDrive/Dataset/

Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).


In [3]:
#name the columns of reviews
df = pd.read_csv("/gdrive/My Drive/Dataset/TxMM_dataset_Process.csv" , sep=',')
x_title = df.values[:,1]  #review.title
x_full = df.values[:,2] #review.text
label = df.values[: , 0]  #review.rating
#creating two differnet classes
label[label != 5] = 0
label[label == 5] = 1
#allocating missing ratings to class1 
NN = []
for i in range(len(label)):
  try:
    (int)(label[i])
    continue
  except:
    NN.append(i)

label[NN] = 1

## Load word2vec-google-news-300 pretrained model
this may take a few minutes because the size of model is about 1.6 GB

In [4]:
nlp = gensim_api.load("word2vec-google-news-300")

## PreProcessing

In [5]:
from nltk.corpus import stopwords

nltk.download('stopwords')
#stopwords.words('english')

stemmer=nltk.PorterStemmer()


corpus_full = []
i = 0
for str_full in x_full:
   if pd.isna(x_title[i]):
     x_title[i] = ''
   if pd.isna(str_full):
     str_full = ''
   string = str(x_title[i]) + ' ' + str(x_title[i]) + ' ' + str_full   # emphasizing on title
   
   i = i + 1
   if i % 500 == 1:
    print(str(i))   # to check the progress
   string = [stemmer.stem(word.lower()) for word in string if not word.lower() in set(stopwords.words('english'))]
   string = ' '.join(string)
   lst_words = string.lower().split()
   lst_grams = [" ".join(lst_words[i:i+1]) 
               for i in range(0, len(lst_words), 1)]
   corpus_full.append(lst_grams)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
1
501
1001
1501
2001
2501
3001
3501
4001
4501
5001
5501
6001
6501
7001
7501
8001
8501
9001
9501
10001
10501
11001
11501
12001
12501
13001
13501
14001
14501
15001
15501
16001
16501
17001
17501
18001
18501
19001
19501
20001
20501
21001
21501
22001
22501
23001
23501
24001
24501
25001
25501
26001
26501
27001
27501
28001
28501
29001
29501
30001
30501
31001
31501
32001
32501
33001
33501
34001
34501


In [6]:
df.head()

,reviews.rating,reviews.title,reviews.text
0,5.0,Kindle,This product so far has not disappointed. My c...
1,5.0,very fast,great for beginner or experienced person. Boug...
2,5.0,Beginner tablet for our 9 year old son.,Inexpensive tablet for him to use and learn on...
3,4.0,Good!!!,I've had my Fire HD 8 two weeks now and I love...
4,5.0,Fantastic Tablet for kids,I bought this for my grand daughter when she c...


## tokenize text

In [48]:
tokenizer = kprocessing.text.Tokenizer(lower=True, split=' ', 
                     oov_token="NaN", 
                     filters='!"#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n')
tokenizer.fit_on_texts(corpus_full)
dic_vocabulary = tokenizer.word_index

## create sequence

In [49]:
full_text2seq = tokenizer.texts_to_sequences(corpus_full)

## padding sequence

In [163]:
Dim = 30

X_full = kprocessing.sequence.pad_sequences(full_text2seq, 
                    maxlen=Dim, padding="post", truncating="post")

## start the matrix (length of vocabulary x vector size) with all 0s

In [164]:
embeddings = np.zeros((len(dic_vocabulary)+1, 30))
for word,idx in dic_vocabulary.items():
    ## update the row with vector
    try:
        embeddings[idx] =  nlp[word]
    ## if word not in model then skip and the row stays all 0s
    except:
        pass

In [165]:
N = np.size(X_full,0)
X_Features = np.zeros((N , 30 * Dim))
for i in range(N):
  for j in range(Dim):
    e = embeddings[X_full[i,j]]
    X_Features[i , j * 30 : (j+1) * 30] = e

# Build TF graph

In [166]:
tf.reset_default_graph()

## Create placeholders

In [167]:
X = tf.placeholder(shape=[None, 30, Dim,1], dtype=tf.float32, name="X")
y = tf.placeholder(shape=[None], dtype=tf.int64, name="y")

## Convolutional layer properties

In [168]:
conv1_params = {
    "filters": 32,
    "kernel_size": [5 , 1],
    "strides": 1,
    "padding": "valid",
    "activation": tf.nn.relu,
}
conv2_params = {
    "filters": 4,
    "kernel_size": [5 , 1],
    "strides": 1,
    "padding": "valid",
    "activation": tf.nn.relu,
}

pool_params = {
    "ksize" : [1,3,1,1],
    "strides" : [1,3,1,1],
    "padding" : "VALID"
}

pool_params2 = {
    "ksize" : [1,4,1,1],
    "strides" : [1,4,1,1],
    "padding" : "VALID"
}

## first conv layer

In [169]:
conv1 = tf.layers.conv2d(X, **conv1_params)  # shape(?, 26, 30, 32)
conv2 = tf.layers.conv2d(X, **conv1_params)
conv3 = tf.layers.conv2d(X, **conv1_params)
conv4 = tf.layers.conv2d(X, **conv1_params)

## first max pooling layer

In [170]:
pool1 = tf.layers.batch_normalization(tf.nn.max_pool(conv1, **pool_params)) # shape(?, 8, 30 ,32)
pool2 = tf.layers.batch_normalization(tf.nn.max_pool(conv2, **pool_params))
pool3 = tf.layers.batch_normalization(tf.nn.max_pool(conv3, **pool_params))
pool4 = tf.layers.batch_normalization(tf.nn.max_pool(conv4, **pool_params))

In [171]:
concatLayer = tf.concat([pool1 , pool2 , pool3 , pool4 ] , axis = 1) #shape(?,32,30,32)

## Second conv  layer

In [172]:
secondConv = tf.layers.conv2d(concatLayer, **conv2_params) #shape(?, 28, 30, 4)

## second max pooling layer

In [173]:
secondPool = tf.nn.max_pool(secondConv, **pool_params) #shape(?, 9, 30, 4)

## Third conv layer

In [174]:
thirdConv = tf.layers.conv2d(secondPool,**conv2_params) #shape(?, 5, 30, 4)

## third max pool layer

In [175]:
thirdPool = tf.layers.batch_normalization(tf.nn.max_pool(thirdConv , **pool_params2)) # additive batch_normalization #shape(?, 1, 30, 4)

In [176]:
flattenLayer = tf.layers.flatten(thirdPool) #shape(?, 120)

## Dense layer

In [177]:
def init_weights(shape):
  init_random_dist = tf.truncated_normal(shape , stddev = 0.1)
  return tf.Variable(init_random_dist)

def init_bias(shape):
  init_bias_vals = tf.constant(0.1 , shape = shape)
  return tf.Variable(init_bias_vals)

In [178]:
input_size = (int)(flattenLayer.get_shape()[1])
W = init_weights([input_size , 32])
b = init_bias([32])
dense1 = tf.matmul(flattenLayer , W) + b
relu1 = tf.nn.relu(dense1)

In [179]:
num_class = 2
Wf = init_weights([32 , num_class])
bf = init_bias([num_class])
y_predicted = tf.matmul(relu1 , Wf) + bf

## Lost function

In [244]:
T = tf.one_hot(y, depth = num_class)
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(labels = T , logits = y_predicted,))
loss = cross_entropy + 0.001 * (tf.norm(W) + tf.norm(Wf))

In [245]:
optimizer = tf.train.AdamOptimizer (learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False, name='Adam')
train = optimizer.minimize(loss)

In [246]:
y_pred_argmax = tf.argmax(y_predicted, axis=1)
correct = tf.equal(y, y_pred_argmax)
accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

# Train

In [247]:
init = tf.global_variables_initializer()

In [248]:
def getNextBatch(batch_size):
  Ntr = np.size(Xtr,0)
  idx = np.random.randint(0 , Ntr , batch_size)
  Xbatch = Xtr[idx,:]
  Ybatch = y_train[idx]
  return Xbatch , Ybatch

In [251]:
from sklearn.model_selection import train_test_split
testPercent = 0.30
Xtr, Xts, y_train, y_test = train_test_split(X_Features,label, test_size=testPercent, random_state = 11)
Ntrain = np.size(Xtr,0)
Ntest = np.size(Xts,0)

In [254]:
#CNN
n_epochs = 10
batch_size = 10
n_iterations_per_epoch = Ntrain // batch_size
n_iterations_test = Ntest

with tf.Session() as sess:
  sess.run(init)
  for epoch in range(n_epochs):
    for iteration in range(1, n_iterations_per_epoch + 1):
        X_batch, y_batch = getNextBatch(batch_size)
        # Run the training operation and measure the loss:
        _, loss_train = sess.run(
            [train, loss],
            feed_dict = {X: X_batch.reshape([-1, 30, Dim, 1]),
                         y: y_batch})
        print("\rIteration: {}/{} ({:.1f}%)  Loss: {:.5f}".format(
                  iteration, n_iterations_per_epoch,
                  iteration * 100 / n_iterations_per_epoch,
                  loss_train),
              end="")

    # At the end of each epoch,
    # measure the validation loss and accuracy:
    loss_tests = []
    acc_tests = []
    step = 100
    for iteration in range(0, n_iterations_test,step):
        X_batch = Xts[iteration:iteration+step,:]
        y_batch = y_test[iteration:iteration+step]
        
        loss_test, acc_test = sess.run(
                [loss, accuracy],
                feed_dict = {X: X_batch.reshape([-1, 30, Dim, 1]),
                             y: np.reshape(y_batch,[-1])})
        loss_tests.append(loss_test)
        acc_tests.append(acc_test)
        print("\rEvaluating the model: {}/{} ({:.1f}%)".format(
                  iteration, n_iterations_test,
                  iteration * 100 / n_iterations_test),
              end=" " * 10)
    
    loss_test = np.mean(loss_tests)
    acc_test = np.mean(acc_tests)
    print("\rEpoch: {}  Test accuracy: {:.4f}%  Loss: {:.6f}".format(
        epoch + 1, acc_test * 100, loss_test))
    
  print('Predicting final labels')
  y_pred = []
  for iteration in range(0, n_iterations_test):
        X_batch = Xts[iteration,:]
        y_batch = y_test[iteration]
        
        yp = sess.run(
                [y_pred_argmax],
                feed_dict = {X: X_batch.reshape([-1, 30, Dim, 1])})
        y_pred.append(yp[:])

Epoch: 1  Test accuracy: 69.0039%  Loss: 0.621052
Epoch: 2  Test accuracy: 69.0039%  Loss: 0.620496
Epoch: 3  Test accuracy: 69.0039%  Loss: 0.619761
Epoch: 4  Test accuracy: 69.0039%  Loss: 0.619987
Epoch: 5  Test accuracy: 69.0039%  Loss: 0.619709
Epoch: 6  Test accuracy: 69.0039%  Loss: 0.619552
Epoch: 7  Test accuracy: 69.0039%  Loss: 0.619658
Epoch: 8  Test accuracy: 69.0039%  Loss: 0.620347
Epoch: 9  Test accuracy: 69.0039%  Loss: 0.619947
Epoch: 10  Test accuracy: 69.0039%  Loss: 0.619396
Predicting final labels


In [255]:
#SVM
from sklearn.metrics import classification_report
from sklearn import svm
clf = svm.SVC()
clf.fit(Xtr, y_train.astype(int))
Yp2 = clf.predict(Xts)
print(classification_report(y_test.astype(int),Yp2))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      3223
           1       0.69      1.00      0.82      7175

    accuracy                           0.69     10398
   macro avg       0.35      0.50      0.41     10398
weighted avg       0.48      0.69      0.56     10398



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [256]:
#KNN
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(Xtr, y_train.astype(int))
Yp5 = neigh.predict(Xts)
print(classification_report(y_test.astype(int),Yp5))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00      3223
           1       0.69      1.00      0.82      7175

    accuracy                           0.69     10398
   macro avg       0.35      0.50      0.41     10398
weighted avg       0.48      0.69      0.56     10398



/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [258]:
# KNN + TF-IDF
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(Xtr, y_train.astype(int))
Yp3 = neigh.predict(Xts)
print(classification_report(y_test.astype(int),Yp3))

              precision    recall  f1-score   support

           0       0.46      0.36      0.40      3286
           1       0.73      0.80      0.76      7112

    accuracy                           0.66     10398
   macro avg       0.59      0.58      0.58     10398
weighted avg       0.64      0.66      0.65     10398



In [259]:
# DT + TF-IDF
from sklearn import tree
clf = tree.DecisionTreeClassifier()
clf.fit(Xtr, y_train.astype(int))
Yp4 = clf.predict(Xts)
print(classification_report(y_test.astype(int),Yp4))


              precision    recall  f1-score   support

           0       0.43      0.43      0.43      3286
           1       0.74      0.74      0.74      7112

    accuracy                           0.64     10398
   macro avg       0.58      0.58      0.58     10398
weighted avg       0.64      0.64      0.64     10398

